### Import required libraries

In [28]:
import pandas as pd
from pathlib import Path
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import json
import logging
from datetime import datetime

### Set up basic logging

In [29]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### First, let's create our sample provider data

In [30]:
provider_data = {
    'provider_id': [1001, 1002, 1003],
    'provider_name': ['Alpha Payments', 'Beta Financial', 'Gamma Processing'],
    'month_1_payments': [150000, 225000, 175000],
    'month_2_payments': [165000, 215000, 180000],
    'month_3_payments': [175000, 235000, 165000],
    'month_4_payments': [180000, 245000, 190000],
    'output_file': ['alpha_report.xlsx', 'beta_report.xlsx', 'gamma_report.xlsx']
}

### Create a DataFrame and save it as CSV

In [31]:
df_providers = pd.DataFrame(provider_data)
df_providers.to_csv('provider_data.csv', index=False)
print("Sample provider data created and saved to CSV:")
display(df_providers)

Sample provider data created and saved to CSV:


,provider_id,provider_name,month_1_payments,month_2_payments,month_3_payments,month_4_payments,output_file
0,1001,Alpha Payments,150000,165000,175000,180000,alpha_report.xlsx
1,1002,Beta Financial,225000,215000,235000,245000,beta_report.xlsx
2,1003,Gamma Processing,175000,180000,165000,190000,gamma_report.xlsx


### Execute the SQL Notebook

In [32]:
def execute_sql_notebook(provider_row):
    """
    Execute the SQL notebook with parameters from a provider row
    """
    logging.info(f"Processing provider: {provider_row['provider_name']} (ID: {provider_row['provider_id']})")
    
    # Load the SQL notebook
    with open('sql_example.ipynb') as f:
        nb = nbformat.read(f, as_version=4)
    
    # Prepare the parameter cell
    parameter_code = f"""
provider_id = {provider_row['provider_id']}
provider_name = '{provider_row['provider_name']}'
month_1_payments = {provider_row['month_1_payments']}
month_2_payments = {provider_row['month_2_payments']}
month_3_payments = {provider_row['month_3_payments']}
month_4_payments = {provider_row['month_4_payments']}
"""
    
    # Update the parameters in the notebook
    nb.cells[1].source = parameter_code  # Assuming parameters are in the second cell
    
    # Execute the notebook
    ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
    try:
        # Execute the notebook and capture the output
        ep.preprocess(nb)
        
        # Get the results from the last cell output
        last_cell_output = nb.cells[-1].outputs[0]
        if 'data' in last_cell_output:
            results = last_cell_output.data['application/json']
        else:
            results = json.loads(last_cell_output.text)
            
        return results
        
    except Exception as e:
        logging.error(f"Error executing SQL notebook: {str(e)}")
        raise

### Create excel

In [33]:
def create_excel_report(results, output_file):
    """
    Create Excel report from SQL analysis results
    """
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Write each analysis section to a different sheet
        for sheet_name, df_data in results.items():
            pd.DataFrame(df_data).to_excel(writer, sheet_name=sheet_name, index=False)
    
    logging.info(f"Report generated: {output_file}")
    return output_file

### Main processing loop

In [34]:
def process_providers():
    """
    Main function to process all providers
    """
    logging.info("Starting provider processing")
    
    for idx, row in df_providers.iterrows():
        try:
            # Execute SQL notebook
            results = execute_sql_notebook(row)
            
            # Create Excel report
            create_excel_report(results, row['output_file'])
            
        except Exception as e:
            logging.error(f"Error processing provider {row['provider_name']}: {str(e)}")
            continue
    
    logging.info("Provider processing completed")

### Execute the process

In [35]:
if __name__ == "__main__":
    process_providers()

2024-11-08 18:33:59,883 - INFO - Starting provider processing
2024-11-08 18:33:59,886 - INFO - Processing provider: Alpha Payments (ID: 1001)
2024-11-08 18:34:01,038 - ERROR - Error executing SQL notebook: An error occurred while executing the following cell:
------------------
if __name__ == "__main__":
    # Setup database
    conn = setup_temp_database()
    
    # Run analysis
    results = run_analysis(conn)
    
    # Display results (when running interactively)
    print("\nPayment Summary:")
    display(results['summary'])
    
    print("\nMonthly Trends:")
    display(results['trends'])
    
    print("\nPayment Distribution:")
    display(results['distribution'])
    
    # Close connection
    conn.close()
    
    # This will be the return value that the Python notebook can use
    results
------------------


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[6], l